In [1]:
#import library
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import *
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from keras.preprocessing import text

2022-01-12 15:59:47.294829: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-12 15:59:47.294858: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
Data = pd.read_csv('/home/abdifatah/Ingenieur_Machine_Learning/QueryResults_3.csv',sep=',')
Data.head(5)

,Id,Body,Title,Tags
0,4,<p>I want to use a <code>Track-Bar</code> to c...,How to convert a Decimal to a Double in C#?,<c#><floating-point><type-conversion><double><...
1,9,<p>Given a <code>DateTime</code> representing ...,How do I calculate someone's age based on a Da...,<c#><.net><datetime>
2,11,<p>Given a specific <code>DateTime</code> valu...,Calculate relative time in C#,<c#><datetime><time><datediff><relative-time-s...
3,16,<p>How do you expose a LINQ query as an ASMX w...,Filling a DataSet or a DataTable from a LINQ q...,<c#><linq><web-services><.net-3.5>
4,17,"<p>How do I store binary data in <a href=""http...",Binary Data in MySQL,<mysql><database><binary-data><data-storage>


In [3]:
all_words_Body_df = pd.read_csv('/home/abdifatah/Ingenieur_Machine_Learning/all_words_Body_df.csv',sep=',')

In [4]:
texte = Data.Title+''+Data.Body
texte.head(5)

0    How to convert a Decimal to a Double in C#?<p>...
1    How do I calculate someone's age based on a Da...
2    Calculate relative time in C#<p>Given a specif...
3    Filling a DataSet or a DataTable from a LINQ q...
4    Binary Data in MySQL<p>How do I store binary d...
dtype: object

In [11]:
# preprocessing Functions for Target and Feature

# Tokenization des Tags
def delete_element(liste):
    element_to_delete = '>'
    try:
        while True:
            liste.remove(element_to_delete)
    except ValueError:
        pass
    return liste

def Tokenization_Tags(data):
    x = []
    y = []
    for row in data['Tags']:
        x.append(row.split('<'))
    for i in range(len(x)):
        x[i].remove('')
    data['Tags']=x
    return data
stop_words = stopwords.words('english')

def element_to_delete_2(data):
    element_to_remove = '>'
    x = data['Tags']
    y = []
    for i in range(len(x)):
        for j in range(len(x[i])):
            x[i][j]=x[i][j].replace('>','')
    data['Tags'] = x
    return data


# keeping only the 30 most frequent tag in the data and returning 1 columns per unique tags like multilabel binarizer
def Tags_reducing(data):
    tags = data['Tags']
    unique_tag_word = []
    for row in tags:
        for word in row:
            unique_tag_word.append(word)
    test = pd.DataFrame()
    test['unique_tag_word'] = unique_tag_word
    data_count = test.unique_tag_word.value_counts()
    tags_df = pd.DataFrame(data = np.zeros((len(data),len(data_count[0:30]))),columns=test.unique_tag_word.value_counts()[0:30].sort_values(ascending = False).index)
    a = 0
    while a <4 :
        a = a+1
        for i in range(len(data['Tags'])):
            for word in data['Tags'][i]:
                if word not in tags_df.columns:
                      data['Tags'][i].remove(word)
    empty_list = []
    for i in range(len(data)):
        if data['Tags'][i] == empty_list:
              data = data.drop(i)
    data = data.reset_index()
    for i in range(len(data)):
        for word in data['Tags'][i]:
            tags_df[word][i]=1
    data = data.drop(columns = 'index')
    data = data.reset_index()
    tags_df = tags_df.reset_index()
    data = data.merge(tags_df,on='index',how='left')
    return data

# preprocessing function
def iml_p5_preprocessing_Tags(data):
    data = Tokenization_Tags(data)
    data = element_to_delete_2(data)
    data = Tags_reducing(data)
    #data = data.drop(columns=['Tags'])
    return data

# tokenization "Body"
def Tokenization_texte(data):
    x = []
    tokenizer = nltk.RegexpTokenizer(r'\w+')
    for row in texte:
        x.append(tokenizer.tokenize(row))
    return x

# stemmatization "Body"
def text_stematization(texte):
    stemmer = SnowballStemmer("english")
    stem_word = []
    stem_sentence = []
    for i in range(len(texte)):
        for word in texte[i]:
            stem_word.append(stemmer.stem(word))
        stem_sentence.append(stem_word)
        stem_word = [] 
    return stem_sentence


# lemmatization "Body"
def text_lemmatization(texte):
    lemmatizer = WordNetLemmatizer()
    lem_word = []
    lem_sentence = []
    for i in range(len(texte)):
        for word in texte[i]:
            lem_word.append(lemmatizer.lemmatize(word))
        lem_sentence.append(lem_word)
        lem_word = [] 
    return lem_sentence
    
# removing stopwords "Body"
def delete_stopwords(texte):
    cleaned_sentence = []
    cleaned_texte = []
    for sentence in texte:
        for word in sentence:
            if word not in stopwords.words('english'):
                cleaned_sentence.append(word)
        cleaned_texte.append(cleaned_sentence)
        cleaned_sentence = []
    return cleaned_texte

# removing frequent_words "Body"
def delete_frequent_words(texte,all_words_df):
    
    frequent_words = list(all_words_Body_df.text_words.value_counts().sort_values(ascending=False)[0:100].index)
    to_keep = ['string', 'http', 'java', 'android', 'list']
    for word in to_keep:
        frequent_words.remove(word)
    cleaned_sentence = []
    cleaned_texte = []
    for sentence in texte:
        for word in sentence:
            if word not in frequent_words:
                cleaned_sentence.append(word)
        cleaned_texte.append(cleaned_sentence)
        cleaned_sentence = []
    return cleaned_texte


def text_reconstruction(texte):
    return " ".join([word for word in texte])  

# text_cleaning "Body"
def text_cleaning(texte,all_words_df):
    cleaned_texte = Tokenization_texte(texte)
    cleaned_texte = text_stematization(cleaned_texte)
    cleaned_texte = text_lemmatization(cleaned_texte)
    cleaned_texte = delete_stopwords(cleaned_texte)
    cleaned_texte = delete_frequent_words(cleaned_texte,all_words_df)
    reconstructed_text = []
    for i in range(len(cleaned_texte)):
        reconstructed_sentence = text_reconstruction(cleaned_texte[i])
        reconstructed_text.append(reconstructed_sentence)
        reconstructed_sentence = []
    return reconstructed_text


In [12]:
# applying cleaning operation to the Feature "Body"
cleaned_texte = text_cleaning(texte,all_words_Body_df)

In [13]:
# replacing old features with the cleaned new one
df = Data.copy()
df =df.drop(columns=['Body','Title'])
df['Body']=cleaned_texte

In [14]:
# visualization
df['Body']

0        convert decim doubl track bar form opac lang p...
1        calcul someon age base datetim birthday given ...
2        calcul relat given specif datetim display rela...
3        fill dataset datat linq queri result expo linq...
4        binari mysql store binari http en wikipedia wi...
                               ...                        
49995                          put form put form send form
49996    pas along variabl xmlhttprequest send variabl ...
49997    convert string byte powershel version sha1 utf...
49998    mock privat powermock still mock privat jndi u...
49999    associ versus hash search googl folk recommend...
Name: Body, Length: 50000, dtype: object

In [15]:
# applying cleaning process to the Tags
df = iml_p5_preprocessing_Tags(df)

In [16]:
# visualization
df

,index,Id,Tags,Body,java,c#,javascript,python,jquery,html,...,regex,iphone,bash,objective-c,ios,windows,ruby,eclipse,tsql,database
0,0,4,[c#],convert decim doubl track bar form opac lang p...,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,9,"[c#, .net]",calcul someon age base datetim birthday given ...,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,11,[c#],calcul relat given specif datetim display rela...,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,16,[c#],fill dataset datat linq queri result expo linq...,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,17,"[mysql, database]",binari mysql store binari http en wikipedia wi...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41163,41163,8057565,[mysql],mysql updat inner join queri idea problem mysq...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41164,41164,8054165,[html],put form put form send form,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41165,41165,8064691,"[javascript, html]",pas along variabl xmlhttprequest send variabl ...,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41166,41166,8057607,[java],mock privat powermock still mock privat jndi u...,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# saving the cleaned data for model testing
import joblib
joblib.dump(df ,'cleaned_data.sav')

['cleaned_data.sav']